# Recommender Systemas - Morphing and Meatafeatures

In [1]:
import sys
print(sys.version)

3.8.19 (default, Mar 20 2024, 19:58:24) 
[GCC 11.2.0]


## Requirements

In [3]:
pip install numpy==1.23.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install easydict==1.10

  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6491 sha256=641fddf002f053cea23acbd0e448b4cabd7b7570b93b08e5e3504ede438839c0
  Stored in directory: /home/beatriziarasilva03/.cache/pip/wheels/fe/4e/02/c9c3154e4845bfdbf1fdf344f5a89f16dcbb4f627a908c9974
Successfully built easydict
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install implicit==0.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 27.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 34.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install matplotlib==3.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 8.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install nltk==3.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 872.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 34.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pandas==1.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 64.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install requests==2.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 3.5 MB/s eta 0:00:0000:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully uninstalled urllib3-2.2.2
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.2
    Uninstalling charset-normalizer-3.3.2:
      Successfully uninstalled charset-normalizer-3.3.2
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.27.2 requires requests>=2.31, but 

In [11]:
pip install s3fs==2023.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.8/308.8 kB 33.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install scikit_learn==1.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 28.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install scipy==1.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 42.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install seaborn==0.12.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 2.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tensorboardX==2.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install torch==1.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 29.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install tqdm==4.64.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install nmslib==2.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 67.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 24.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install annoy==1.17.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=533395 sha256=0e9dcfed737971b49694d8cf9b34ff42581eb227539aa64ff1061cddf0455c5c
  Stored in directory: /home/beatriziarasilva03/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install pandarallel==1.6.4

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.3 MB/s eta 0:00:00 0:00:01
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16661 sha256=9cf503e73f54a2d106f94044f252136dfc25a0287309b11f78d33a9a93da28c4
  Stored in directory: /home/beatriziarasilva03/.cache/pip/wheels/62/1e/e7/f9ee096e5cc02890a6934a5670ff6e45a3400f330605bd8210
Successfully built pandarallel
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install recommenders[examples]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.1 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 58.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
pip install cornac

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install scrapbook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 37.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install papermill

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 2.0 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Imports

In [2]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as pltt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix

In [3]:
from recommendation import implicit_util
import implicit.evaluation
from exploration import exploration_util

## MetaLearning

### Space D - Building Trajectories

In [4]:
def traj_1(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=1))

def traj_2(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=2))

def traj_3(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=3))

def traj_4(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=4))

def traj_5(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=5))

def traj_6(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=6))

def traj_7(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=7))

def traj_8(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=8))

def traj_9(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=9))

def traj_10(df_source, df_target):
    return list(swap_rows(df_source, df_target, seed=10))

In [5]:
def swap_rows(df_source, df_target, seed=None):
    np.random.seed(seed)
    # Obter uma permutação aleatória dos índices das linhas
    indices = np.random.permutation(len(df_source))
    df_intermediate = df_source.copy()
    for idx in indices:
        df_intermediate.iloc[idx] = df_target.iloc[idx]
        yield df_intermediate.copy()

### Space MF - Extracting MetaFeatures

In [6]:
from scipy.stats import entropy, kurtosis
def gini(x):
    #fonte: https://stackoverflow.com/a/39513799
    #calcula o indice de gini normalizado
    mad = np.abs(np.subtract.outer(x, x)).mean()
    rmad = mad / np.mean(x)
    g = 0.5 * rmad
    return g
    
def calculate_metafeatures(df):
    metafeatures = {}
    column_counts = df.count(axis=0).to_numpy()
    metafeatures['column.count.entropy'] = entropy(df.count(axis=0))
    metafeatures['column.count.gini'] = gini(column_counts)
    #metafeatures['column.count.kurtosis'] = kurtosis(df.count(axis=0))
    metafeatures['column.count.mean'] = np.mean(df.count(axis=0))
    metafeatures['column.mean.entropy'] = entropy(df.mean(axis=0))
    metafeatures['row.count.entropy'] = entropy(df.count(axis=1))
    #metafeatures['row.count.kurtosis'] = kurtosis(df.count(axis=1))
    metafeatures['row.count.max'] = np.max(df.count(axis=1))
    metafeatures['nrBin'] = np.sum(df.nunique())
    metafeatures['attrConc.mean'] = np.mean(df.var(axis=0))
    #metafeatures['attrEnt.mean'] = np.mean(entropy(df, axis=0))
    metafeatures['nZeros'] = np.sum(df == 0)
    metafeatures['sparsity'] = np.sum(df == 0) / df.size
    
    return pd.DataFrame(metafeatures, index=[0])

In [39]:
def metafeatures_data(dfs):
    metafeatures_list = []
    for df in dfs:
        df_wide = df.pivot(index='new_member_id', columns='new_recipe_id', values='rating')
        metafeatures_df = calculate_metafeatures(df_wide)
        metafeatures_list.append(metafeatures_df)
    combined_metafeatures_df = pd.concat(metafeatures_list, ignore_index=True)
    return combined_metafeatures_df

### Space F - Extracting MetaLabels

In [8]:
def evaluation(df):
    #Construçao da matrix (user x recipe x ratings)
    user_recipe_matrix = csr_matrix((df['rating'], (df['new_member_id'], df['new_recipe_id'])))
    # Test/train split #Alternatively use implicit.evaluation.leave_k_out_split to force each user being in both sets
    train_matrix, test_matrix = implicit.evaluation.train_test_split(user_recipe_matrix.tocsr().tocoo())
    
    # Get users/recipes in the train set (or test set respectively)
    train_user, train_recipe = implicit_util.tuple_to_unique(train_matrix.tocsr().nonzero())
    test_user, test_recipe = implicit_util.tuple_to_unique(test_matrix.tocsr().nonzero())
    
    # Executes all models, exception on Windows/Python3.10: nmslib_als, faiss_als
    evaluation, recommendations, similar_items, similar_users = implicit_util.train_and_execute_all(train_matrix, test_matrix, train_user, train_recipe, ['nmslib_als', 'faiss_als'], K=10)
    return evaluation

In [9]:
def melhor_alg(df):
    #eval_df = pd.DataFrame(df)
    df['media']=df.mean(axis=1) #faz a media dos valores das metricas para cada alg
    melhor_algoritmo = df['media'].idxmax()
    df['melhor_algoritmo']=df.index==melhor_algoritmo
    return melhor_algoritmo

In [10]:
def obter_metalabels(dfs):
    labels=[]
    for df in dfs:
        evaluations=evaluation(df)
        label=melhor_alg(evaluations)
        labels.append(label)
    return labels

In [11]:
def metados(df,labels):
    df['Algoritmo'] = labels
    return df

### MetaDataSet

In [12]:
def df_MetaFeatures(Ds, Dt):
    #Trajetorias
    intermediates_1 = traj_1(Ds, Dt)
    '''intermediates_2 = traj_2(Ds, Dt)
    intermediates_3 = traj_3(Ds, Dt)
    intermediates_4 = traj_4(Ds, Dt)
    intermediates_5 = traj_5(Ds, Dt)
    intermediates_6 = traj_6(Ds, Dt)
    intermediates_7 = traj_7(Ds, Dt)
    intermediates_8 = traj_8(Ds, Dt)
    intermediates_9 = traj_9(Ds, Dt)
    intermediates_10 = traj_10(Ds, Dt)'''
    
    #Calcular Metafeatures
    meafeatures_intermedio1=metafeatures_data(intermediates_1)
    '''meafeatures_intermedio2=metafeatures_data(intermediates_2)
    meafeatures_intermedio3=metafeatures_data(intermediates_3)
    meafeatures_intermedio4=metafeatures_data(intermediates_4)
    meafeatures_intermedio5=metafeatures_data(intermediates_5)
    meafeatures_intermedio6=metafeatures_data(intermediates_6)
    meafeatures_intermedio7=metafeatures_data(intermediates_7)
    meafeatures_intermedio8=metafeatures_data(intermediates_8)
    meafeatures_intermedio9=metafeatures_data(intermediates_9)
    meafeatures_intermedio10=metafeatures_data(intermediates_10)'''

    #Dataset MetaFeatures sem Metalabels
    df_metafeatures = pd.concat([meafeatures_intermedio1, meafeatures_intermedio2, meafeatures_intermedio3,meafeatures_intermedio4,meafeatures_intermedio5,meafeatures_intermedio6,meafeatures_intermedio7,meafeatures_intermedio8,meafeatures_intermedio9,meafeatures_intermedio10], ignore_index=True)
    
    #Metalabels
    labels_intermedio1=obter_metalabels(intermediates_1)
    labels_intermedio2=obter_metalabels(intermediates_2)
    labels_intermedio3=obter_metalabels(intermediates_3)
    labels_intermedio4=obter_metalabels(intermediates_4)
    labels_intermedio5=obter_metalabels(intermediates_5)
    labels_intermedio6=obter_metalabels(intermediates_6)
    labels_intermedio7=obter_metalabels(intermediates_7)
    labels_intermedio8=obter_metalabels(intermediates_8)
    labels_intermedio9=obter_metalabels(intermediates_9)
    labels_intermedio10=obter_metalabels(intermediates_10)
    
    #juntar labels numa so
    df1 = pd.DataFrame(labels_intermedio1, columns=['label'])
    df2 = pd.DataFrame(labels_intermedio2, columns=['label'])
    df3 = pd.DataFrame(labels_intermedio3, columns=['label'])
    df4 = pd.DataFrame(labels_intermedio4, columns=['label'])
    df5 = pd.DataFrame(labels_intermedio5, columns=['label'])
    df6 = pd.DataFrame(labels_intermedio6, columns=['label'])
    df7 = pd.DataFrame(labels_intermedio7, columns=['label'])
    df8 = pd.DataFrame(labels_intermedio8, columns=['label'])
    df9 = pd.DataFrame(labels_intermedio9, columns=['label'])
    df10 = pd.DataFrame(labels_intermedio10, columns=['label'])

    # Junta todos os dataframes em um só
    df_total = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index=True)
    
    #Dataset MetaFeatures com Metalabels
    if (len(df_metafeatures) != len(df_total)):
                         raise ValueError("O número de linhas de df_metafeatures e df_total deve ser o mesmo")
    df_metafeatures['Alg'] = df_total['label'].values

    #Normalizar
    scaler = MinMaxScaler()
    numeric_cols = df_metafeatures.select_dtypes(include='number').columns
    df_metafeatures[numeric_cols] = scaler.fit_transform(df_metafeatures[numeric_cols])
    
    
    return df_metafeatures

## MetaFeatures - Internal Trajectories

In [71]:
import os
print(os.getcwd())

/home/beatriziarasilva03/Estagio


In [72]:
dir='/home/beatriziarasilva03/Estagio/data'

In [73]:
hummus_reviews=pd.read_csv(dir+'/pp_reviews.csv')

/tmp/ipykernel_1201/4171652366.py:1: DtypeWarning: Columns (4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  hummus_reviews=pd.read_csv(dir+'/pp_reviews.csv')


In [74]:
hummus_reviews.shape

(1916424, 12)

In [75]:
#hummus_reviews=df_sorted = hummus_reviews.sort_values(by='new_member_id')

In [76]:
df_reduzido = hummus_reviews.sample(n=520, random_state=42)

In [77]:
df_reduzido.shape

(520, 12)

In [78]:
df_reduzido.columns

Index(['Unnamed: 0', 'member_id', 'recipe_id', 'rating', 'text', 'likes',
       'last_modified_date', 'review_id', 'review_url', 'member_url',
       'new_member_id', 'new_recipe_id'],
      dtype='object')

In [79]:
reviews=df_reduzido[['new_member_id','new_recipe_id','rating']]

In [80]:
reviews.shape

(520, 3)

In [81]:
reviews

,new_member_id,new_recipe_id,rating
1644814,14998,235282,6.0
185951,3977,341551,5.0
4208,2723,276466,3.0
455312,289,443133,4.0
750722,11279,48617,5.0
...,...,...,...
178754,673,339059,5.0
1410464,152060,551,6.0
1450273,67641,40410,6.0
1410990,9398,1078,6.0


In [82]:
df_wide = reviews.pivot(index='new_member_id', columns='new_recipe_id', values='rating')
df_wide

new_recipe_id,551,1078,2024,2292,2480,3655,3986,3987,9051,9359,...,485196,493556,494032,498168,498325,498813,503585,505535,505908,506750
new_member_id,,,,,,,,,,,,,,,,,,,,,
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
Ds = reviews.sample(frac=0.5, random_state=42)  
Dt = reviews.drop(Ds.index)

In [63]:
print(Ds.shape, Dt.shape)

(260, 3) (260, 3)


In [64]:
intermediates_1 = traj_1(Ds, Dt)

In [65]:
meafeatures_intermedio1=metafeatures_data(intermediates_1)

In [66]:
meafeatures_intermedio1

,column.count.entropy,column.count.gini,column.count.mean,column.mean.entropy,row.count.entropy,row.count.max,nrBin,attrConc.mean,nZeros,sparsity
0,5.550018,0.007633,1.007752,5.532852,5.508670,3,258,0.0,NaN,NaN
1,5.550018,0.007633,1.007752,5.532870,5.501325,3,258,0.0,NaN,NaN
2,5.550018,0.007633,1.007752,5.532800,5.501325,3,258,0.0,NaN,NaN
3,5.550018,0.007633,1.007752,5.533110,5.501325,3,258,0.0,NaN,NaN
4,5.550018,0.007633,1.007752,5.533729,5.501325,3,258,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
255,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,NaN,NaN,NaN
256,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,NaN,NaN,NaN
257,5.560682,0.000000,1.000000,5.529528,5.510682,3,260,NaN,NaN,NaN
258,5.560682,0.000000,1.000000,5.529518,5.510682,3,260,NaN,NaN,NaN


In [67]:
labels_intermedio1=obter_metalabels(intermediates_1)

getting model als


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

...

  0%|          | 0/506751 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

Current model does not support the method similar_users()


In [68]:
df1 = pd.DataFrame(labels_intermedio1, columns=['label'])

In [69]:
meafeatures_intermedio1['Alg'] = df1['label'].values

In [70]:
meafeatures_intermedio1

,column.count.entropy,column.count.gini,column.count.mean,column.mean.entropy,row.count.entropy,row.count.max,nrBin,attrConc.mean,nZeros,sparsity,Alg
0,5.550018,0.007633,1.007752,5.532852,5.508670,3,258,0.0,NaN,NaN,als
1,5.550018,0.007633,1.007752,5.532870,5.501325,3,258,0.0,NaN,NaN,als
2,5.550018,0.007633,1.007752,5.532800,5.501325,3,258,0.0,NaN,NaN,als
3,5.550018,0.007633,1.007752,5.533110,5.501325,3,258,0.0,NaN,NaN,als
4,5.550018,0.007633,1.007752,5.533729,5.501325,3,258,0.0,NaN,NaN,als
...,...,...,...,...,...,...,...,...,...,...,...
255,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,NaN,NaN,NaN,als
256,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,NaN,NaN,NaN,als
257,5.560682,0.000000,1.000000,5.529528,5.510682,3,260,NaN,NaN,NaN,als
258,5.560682,0.000000,1.000000,5.529518,5.510682,3,260,NaN,NaN,NaN,als


In [49]:
# Verifica se há alguma duplicata no DataFrame
has_duplicates = meafeatures_intermedio1.duplicated().any()
print(f"Existem duplicatas no DataFrame? {has_duplicates}")

# Conta o número de duplicatas no DataFrame
num_duplicates = meafeatures_intermedio1.duplicated().sum()
print(f"Número de duplicatas no DataFrame: {num_duplicates}")

Existem duplicatas no DataFrame? True
Número de duplicatas no DataFrame: 72


In [83]:
unique_algorithms = meafeatures_intermedio1['Alg'].unique()
unique_algorithms

array(['als', 'bpr'], dtype=object)

In [85]:
alg_counts = meafeatures_intermedio1['Alg'].value_counts()
alg_counts

Alg
als    248
bpr     12
Name: count, dtype: int64

In [87]:
null_counts = meafeatures_intermedio1.isnull().sum()
null_counts

column.count.entropy      0
column.count.gini         0
column.count.mean         0
column.mean.entropy       0
row.count.entropy         0
row.count.max             0
nrBin                     0
attrConc.mean            19
nZeros                  260
sparsity                260
Alg                       0
dtype: int64

In [88]:
columns_to_drop = ['nZeros', 'sparsity']
meafeatures_intermedio1 = meafeatures_intermedio1.drop(columns=columns_to_drop)

In [89]:
mean_rating = meafeatures_intermedio1['attrConc.mean'].mean()
meafeatures_intermedio1['attrConc.mean'].fillna(mean_rating, inplace=True)

In [90]:
meafeatures_intermedio1

,column.count.entropy,column.count.gini,column.count.mean,column.mean.entropy,row.count.entropy,row.count.max,nrBin,attrConc.mean,Alg
0,5.550018,0.007633,1.007752,5.532852,5.508670,3,258,0.000000,als
1,5.550018,0.007633,1.007752,5.532870,5.501325,3,258,0.000000,als
2,5.550018,0.007633,1.007752,5.532800,5.501325,3,258,0.000000,als
3,5.550018,0.007633,1.007752,5.533110,5.501325,3,258,0.000000,als
4,5.550018,0.007633,1.007752,5.533729,5.501325,3,258,0.000000,als
...,...,...,...,...,...,...,...,...,...
255,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,0.721914,als
256,5.560682,0.000000,1.000000,5.529454,5.510682,3,260,0.721914,als
257,5.560682,0.000000,1.000000,5.529528,5.510682,3,260,0.721914,als
258,5.560682,0.000000,1.000000,5.529518,5.510682,3,260,0.721914,als


## Model

In [94]:
X = meafeatures_intermedio1.drop(columns=['Alg'])
y = meafeatures_intermedio1['Alg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9230769230769231


**Otimizar codigo**

In [52]:

def gini(x):
    # fonte: https://stackoverflow.com/a/39513799
    mad = np.abs(np.subtract.outer(x, x)).mean()
    rmad = mad / np.mean(x)
    g = 0.5 * rmad
    return g

def calculate_metafeatures(df):
    metafeatures = {}
    column_counts = df.count(axis=0).to_numpy()
    metafeatures['column.count.entropy'] = entropy(df.count(axis=0))
    metafeatures['column.count.gini'] = gini(column_counts)
    # metafeatures['column.count.kurtosis'] = kurtosis(df.count(axis=0))
    metafeatures['column.count.mean'] = np.mean(df.count(axis=0))
    metafeatures['column.mean.entropy'] = entropy(df.mean(axis=0))
    metafeatures['row.count.entropy'] = entropy(df.count(axis=1))
    # metafeatures['row.count.kurtosis'] = kurtosis(df.count(axis=1))
    metafeatures['row.count.max'] = np.max(df.count(axis=1))
    metafeatures['nrBin'] = np.sum(df.nunique())
    metafeatures['attrConc.mean'] = np.mean(df.var(axis=0))
    # metafeatures['attrEnt.mean'] = np.mean(entropy(df, axis=0))
    metafeatures['nZeros'] = np.sum(df == 0)
    metafeatures['sparsity'] = np.sum(df == 0) / df.size
    
    return pd.DataFrame(metafeatures, index=[0])

def swap_rows_and_calculate_metafeatures(df_source, df_target, seed=None):
    np.random.seed(seed)
    indices = np.random.permutation(len(df_source))
    df_intermediate = df_source.copy()
    for idx in indices:
        metafeatures_df = calculate_metafeatures(df_intermediate)
        evaluations = evaluation(df_intermediate)
        metalabel = melhor_alg(evaluations)
        metafeatures_df['metalabel'] = metalabel
        yield metafeatures_df

def traj_1_1(df_source, df_target):
    return list(swap_rows_and_calculate_metafeatures(df_source, df_target, seed=1))

def traj_1_2(df_source, df_target):
    return list(swap_rows_and_calculate_metafeatures(df_source, df_target, seed=2))

def traj_1_3(df_source, df_target):
    return list(swap_rows_and_calculate_metafeatures(df_source, df_target, seed=3))

def traj_1_4(df_source, df_target):
    return list(swap_rows_and_calculate_metafeatures(df_source, df_target, seed=4))

def metafeatures_data(dfs):
    return pd.concat(dfs, ignore_index=True)